In [1]:
import pandas as pd
import numpy as np
import re
data1=pd.read_csv('DATA/laptop_data.csv')

# Data1

In [2]:
len(data1)

1303

In [3]:
data1.head()

,Unnamed: 0,Company,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price
0,0,Apple,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 2.3GHz,8GB,128GB SSD,Intel Iris Plus Graphics 640,macOS,1.37kg,71378.6832
1,1,Apple,Ultrabook,13.3,1440x900,Intel Core i5 1.8GHz,8GB,128GB Flash Storage,Intel HD Graphics 6000,macOS,1.34kg,47895.5232
2,2,HP,Notebook,15.6,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8GB,256GB SSD,Intel HD Graphics 620,No OS,1.86kg,30636.0000
3,3,Apple,Ultrabook,15.4,IPS Panel Retina Display 2880x1800,Intel Core i7 2.7GHz,16GB,512GB SSD,AMD Radeon Pro 455,macOS,1.83kg,135195.3360
4,4,Apple,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 3.1GHz,8GB,256GB SSD,Intel Iris Plus Graphics 650,macOS,1.37kg,96095.8080


In [4]:
data1.columns

Index(['Unnamed: 0', 'Company', 'TypeName', 'Inches', 'ScreenResolution',
       'Cpu', 'Ram', 'Memory', 'Gpu', 'OpSys', 'Weight', 'Price'],
      dtype='object')

In [5]:
data1.drop('TypeName', axis=1, inplace=True)

In [6]:
data1.drop('ScreenResolution', axis=1, inplace=True)

In [7]:
data1.drop('OpSys', axis=1, inplace=True)

In [8]:
data1.drop('Unnamed: 0', axis=1, inplace=True)

In [9]:
data1['Ram'] = data1['Ram'].str.replace('GB','')
data1['Weight'] = data1['Weight'].str.replace('kg','')
data1['Ram'] = data1['Ram'].astype('int32')
data1['Weight'] = data1['Weight'].astype('float32')

In [10]:
data1['Ram'].value_counts()

Ram
8     619
4     375
16    200
6      41
12     25
2      22
32     17
24      3
64      1
Name: count, dtype: int64

In [11]:
data1['Memory'].value_counts()

Memory
256GB SSD                        412
1TB HDD                          223
500GB HDD                        132
512GB SSD                        118
128GB SSD +  1TB HDD              94
128GB SSD                         76
256GB SSD +  1TB HDD              73
32GB Flash Storage                38
2TB HDD                           16
64GB Flash Storage                15
512GB SSD +  1TB HDD              14
1TB SSD                           14
256GB SSD +  2TB HDD              10
1.0TB Hybrid                       9
256GB Flash Storage                8
16GB Flash Storage                 7
32GB SSD                           6
180GB SSD                          5
128GB Flash Storage                4
512GB SSD +  2TB HDD               3
16GB SSD                           3
512GB Flash Storage                2
1TB SSD +  1TB HDD                 2
256GB SSD +  500GB HDD             2
128GB SSD +  2TB HDD               2
256GB SSD +  256GB SSD             2
512GB SSD +  256GB SSD         

In [12]:
data1['STOCKAGESSD'] = None
data1['STOCKAGEHDD'] = None
data1['STOCKAGEFlash'] = None

# Define a function to extract memory values
def extract_memory(row):
    if 'Hybrid' in row['Memory']:
        return row
    if 'SSD' in row['Memory']:
        row['STOCKAGESSD'] = re.search(r'(\d+)(GB|TB) SSD', row['Memory']).group()
    if 'HDD' in row['Memory']:
        row['STOCKAGEHDD'] = re.search(r'(\d+)(GB|TB) HDD', row['Memory']).group()
    if 'Flash Storage' in row['Memory']:
        row['STOCKAGEFlash'] = re.search(r'(\d+)(GB|TB) Flash Storage', row['Memory']).group()
    return row
data1 = data1.apply(extract_memory, axis=1)

# Drop rows with 'Hybrid' in 'Memory'
data1 = data1[~data1['Memory'].str.contains('Hybrid')]

In [13]:
data1[['STOCKAGEHDD', 'STOCKAGESSD', 'STOCKAGEFlash']] = data1[['STOCKAGEHDD', 'STOCKAGESSD', 'STOCKAGEFlash']].fillna(0)
len(data1)

1291

Now that we have completed the Memory column, we will proceed to STOCKAGE HDD, STOCKAGE SSD, STOCKAGE Flash, and prepare them

In [14]:
data1.drop('Memory', axis=1, inplace=True)

In [15]:
def clean_memory(row):
    if re.search(r'TB',str(row['STOCKAGESSD'])):
        row['STOCKAGESSD']=re.sub(r' SSD','',str(row['STOCKAGESSD']))
        row['STOCKAGESSD']=float(re.sub(r'TB','',str(row['STOCKAGESSD'])))*1000
    if re.search(r'TB',str(row['STOCKAGEHDD'])):
        row['STOCKAGEHDD']=re.sub(r' HDD','',str(row['STOCKAGEHDD']))
        row['STOCKAGEHDD']=float(re.sub(r'TB','',str(row['STOCKAGEHDD'])))*1000
    if re.search(r'TB',str(row['STOCKAGEFlash'])):
        row['STOCKAGEFlash']=re.sub(r' Flash Storage','',str(row['STOCKAGEFlash']))
        row['STOCKAGEFlash']=float(re.sub(r'TB','',str(row['STOCKAGEFlash'])))*1000
    if re.search(r'GB',str(row['STOCKAGESSD'])):
        row['STOCKAGESSD']=re.sub(r' SSD','',str(row['STOCKAGESSD']))
        row['STOCKAGESSD']=float(re.sub(r'GB','',str(row['STOCKAGESSD'])))
    if re.search(r'GB',str(row['STOCKAGEHDD'])):
        row['STOCKAGEHDD']=re.sub(r' HDD','',str(row['STOCKAGEHDD']))
        row['STOCKAGEHDD']=float(re.sub(r'GB','',str(row['STOCKAGEHDD'])))
    if re.search(r'GB',str(row['STOCKAGEFlash'])):
        row['STOCKAGEFlash']=re.sub(r' Flash Storage','',str(row['STOCKAGEFlash']))
        row['STOCKAGEFlash']=float(re.sub(r'GB','',str(row['STOCKAGEFlash'])))
    return row
data1 = data1.apply(clean_memory, axis=1)

In [16]:
count_zero_ssd = (data1['STOCKAGESSD'] != 0).sum()

# Display the count
print(f"Number of rows where STOCKAGESSD is not 0: {count_zero_ssd}")

Number of rows where STOCKAGESSD is not 0: 841


In [17]:
count_zero_ssd = (data1['STOCKAGEHDD'] != 0).sum()

# Display the count
print(f"Number of rows where STOCKAGEHDD is not 0: {count_zero_ssd}")

Number of rows where STOCKAGEHDD is not 0: 575


In [18]:
count_zero_ssd = (data1['STOCKAGEFlash'] != 0).sum()

# Display the count
print(f"Number of rows where STOCKAGEFlash is not 0: {count_zero_ssd}")

Number of rows where STOCKAGEFlash is not 0: 75


In [19]:
data1.head()

,Company,Inches,Cpu,Ram,Gpu,Weight,Price,STOCKAGESSD,STOCKAGEHDD,STOCKAGEFlash
0,Apple,13.3,Intel Core i5 2.3GHz,8,Intel Iris Plus Graphics 640,1.37,71378.6832,128.0,0.0,0.0
1,Apple,13.3,Intel Core i5 1.8GHz,8,Intel HD Graphics 6000,1.34,47895.5232,0.0,0.0,128.0
2,HP,15.6,Intel Core i5 7200U 2.5GHz,8,Intel HD Graphics 620,1.86,30636.0000,256.0,0.0,0.0
3,Apple,15.4,Intel Core i7 2.7GHz,16,AMD Radeon Pro 455,1.83,135195.3360,512.0,0.0,0.0
4,Apple,13.3,Intel Core i5 3.1GHz,8,Intel Iris Plus Graphics 650,1.37,96095.8080,256.0,0.0,0.0


In [20]:
mark_counts = data1['Company'].value_counts()
data1 = data1[data1['Company'].isin(mark_counts.index[mark_counts >= 10])]
data1 = pd.get_dummies(data1, columns=['Company'], prefix='')

In [21]:
data1['CPU_Brand'] = None
data1['CPU_Modifier'] = None
data1['CPU_Generation'] = None

# Define regular expressions

# Apply the function to each row
def extract_cpu_info(row):
    if re.search(r'Intel',row['Cpu']):
        row['CPU_Brand']='Intel'
        if re.search(r'((\d{4,6})|(\d{1,2}th)|(\d{1,2}(e|è|é)me)|(i\d-?\d{4,6}?))',row['Cpu']):
            generation=re.search(r'((\d{4,6})|(\d{1,2}th)|(\d{1,2}(e|è|é)me)|(i\d-?\d{4,6}?))',row['Cpu']).group()
            row['CPU_Generation']=generation
            if generation[0] in ['3', '4', '5', '6', '7', '8', '9']:
                row['CPU_Generation'] = int(generation[0])
            else:
                row['CPU_Generation'] = int(generation[0:2])
        if re.search(r'i[3579]',row['Cpu']):
            row['CPU_Modifier']=re.search(r'i[3579]',row['Cpu']).group(0)[1]
        
    elif re.search(r'AMD',row['Cpu']):
        row['CPU_Brand']='AMD'
        if re.search(r'\d{4,5}(u|r|x)',row['Cpu']):
            row['CPU_Modifier']=re.search(r'\d{4,5}(u|r|x)',row['Cpu']).group(0)[0]
            if re.search(r'\d{4,5}(u|r|x)',row['Cpu']).group(0)[0] in ['3', '4', '5', '6', '7', '8', '9']:
                row['CPU_Generation'] =  re.search(r'\d{4,5}(u|r|x)',row['Cpu']).group(0)[0]
            else:
                row['CPU_Generation'] = re.search(r'\d{4,5}(u|r|x)',row['Cpu']).group(0)[0:2]
    return row
data1 = data1.apply(extract_cpu_info, axis=1)

In [22]:
data1['Cpu'].value_counts().sample(60)

Cpu
Intel Core i5 3.1GHz                       2
AMD E-Series E2-9000e 1.5GHz               2
Intel Celeron Dual Core N3060 1.60GHz      1
AMD A6-Series 9220 2.5GHz                  7
Intel Core i7 2.8GHz                       1
Intel Core i5 2.9GHz                       1
Intel Core i5 7300HQ 2.5GHz               32
AMD E-Series E2-6110 1.5GHz                1
Intel Pentium Quad Core N4200 1.1GHz      14
Intel Core i7 6500U 2.50GHz                2
AMD A12-Series 9720P 2.7GHz                2
AMD E-Series 7110 1.8GHz                   2
Intel Core i3 6006U 2.0GHz                16
Intel Core i3 6006U 2GHz                  64
Intel Core M 6Y54 1.1GHz                   1
Intel Core i7 8650U 1.9GHz                 2
Intel Core i7 6820HQ 2.7GHz                7
Intel Core i7 6600U 2.6GHz                18
Intel Pentium Quad Core N3710 1.6GHz      11
Intel Core i7 2.2GHz                       1
Intel Core i7 7700HQ 2.8GHz              141
Intel Core i5 7200U 2.5GHz               183
Intel 

In [23]:
data1 = data1.drop(data1[(data1['CPU_Brand'].isna()) & (data1['CPU_Modifier'].isna()) & (data1['CPU_Generation'].isna())].index)

# Reset index after dropping rows
data1 = data1.reset_index(drop=True)
data1 = pd.get_dummies(data1, columns=['CPU_Brand'], prefix='CPU')

In [24]:
def convert_price(value):
    return value*0.12
data1['Price']=data1['Price'].apply(convert_price)

In [25]:
len(data1)

1240

In [26]:
data1.head()

,Inches,Cpu,Ram,Gpu,Weight,Price,STOCKAGESSD,STOCKAGEHDD,STOCKAGEFlash,_Acer,...,_Asus,_Dell,_HP,_Lenovo,_MSI,_Toshiba,CPU_Modifier,CPU_Generation,CPU_AMD,CPU_Intel
0,13.3,Intel Core i5 2.3GHz,8,Intel Iris Plus Graphics 640,1.37,8565.441984,128.0,0.0,0.0,False,...,False,False,False,False,False,False,5,NaN,False,True
1,13.3,Intel Core i5 1.8GHz,8,Intel HD Graphics 6000,1.34,5747.462784,0.0,0.0,128.0,False,...,False,False,False,False,False,False,5,NaN,False,True
2,15.6,Intel Core i5 7200U 2.5GHz,8,Intel HD Graphics 620,1.86,3676.320000,256.0,0.0,0.0,False,...,False,False,True,False,False,False,5,7.0,False,True
3,15.4,Intel Core i7 2.7GHz,16,AMD Radeon Pro 455,1.83,16223.440320,512.0,0.0,0.0,False,...,False,False,False,False,False,False,7,NaN,False,True
4,13.3,Intel Core i5 3.1GHz,8,Intel Iris Plus Graphics 650,1.37,11531.496960,256.0,0.0,0.0,False,...,False,False,False,False,False,False,5,NaN,False,True


In [27]:
data1.columns

Index(['Inches', 'Cpu', 'Ram', 'Gpu', 'Weight', 'Price', 'STOCKAGESSD',
       'STOCKAGEHDD', 'STOCKAGEFlash', '_Acer', '_Apple', '_Asus', '_Dell',
       '_HP', '_Lenovo', '_MSI', '_Toshiba', 'CPU_Modifier', 'CPU_Generation',
       'CPU_AMD', 'CPU_Intel'],
      dtype='object')

In [28]:
data1.drop('Cpu', axis=1, inplace=True)

In [29]:
data1=data1.rename(columns={'Gpu':'GPU','Weight':'POIDS','Inches':'ECRAN','Price':'PRIX','Ram':'RAM'})

In [30]:
data1['GPU'].value_counts()

GPU
Intel HD Graphics 620       267
Intel HD Graphics 520       181
Intel UHD Graphics 620       68
Nvidia GeForce GTX 1050      63
Nvidia GeForce GTX 1060      45
                           ... 
Intel HD Graphics 540         1
AMD R4 Graphics               1
Nvidia GeForce GTX 1070M      1
Nvidia GeForce GTX1060        1
Nvidia GeForce 960M           1
Name: count, Length: 107, dtype: int64

In [31]:

data1.loc[data1['GPU'].str.contains('Intel')  & data1['GPU'].str.contains('6000'), 'GPU'] = 'Intel HD Graphics 620'
data1.loc[data1['GPU'].str.contains('Intel')  & data1['GPU'].str.contains('615'), 'GPU'] = 'Intel HD Graphics 620'
data1.loc[data1['GPU'].str.contains('Intel')  & data1['GPU'].str.contains('620'), 'GPU'] = 'Intel HD Graphics 620'
data1.loc[data1['GPU'].str.contains('Intel')  & data1['GPU'].str.contains('630'), 'GPU'] = 'Intel HD Graphics 620'
data1.loc[data1['GPU'].str.contains('Intel')  & (data1['GPU'].str.contains('400')|data1['GPU'].str.contains('405')), 'GPU'] = 'Intel HD Graphics 400'
data1.loc[data1['GPU'].str.contains('Nvidia') & data1['GPU'].str.contains('GTX'), 'GPU'] = 'Nvidia  GTX'
data1.loc[data1['GPU'].str.contains('Nvidia') & data1['GPU'].str.contains('RTX'), 'GPU'] = 'Nvidia  RTX'
data1.loc[data1['GPU'].str.contains('Nvidia') & data1['GPU'].str.contains('MX'), 'GPU'] = 'Nvidia  MX'
data1.loc[data1['GPU'].str.contains('Nvidia') & data1['GPU'].str.contains('M') & ~(data1['GPU'].str.contains('MX')) , 'GPU'] = 'Nvidia  M'
data1.loc[data1['GPU'].str.contains('Nvidia') & data1['GPU'].str.contains('Quadro'), 'GPU'] = 'Nvidia  Quadro'
data1.loc[data1['GPU'].str.contains('Intel')  & data1['GPU'].str.contains('HD') & (data1['GPU'].str.contains('520')|data1['GPU'].str.contains('5200')|data1['GPU'].str.contains('515')|data1['GPU'].str.contains('505')|data1['GPU'].str.contains('500')|data1['GPU'].str.contains('510')), 'GPU'] = 'Intel HD Graphics 520'
data1.loc[data1['GPU'].str.contains('Intel')  & data1['GPU'].str.contains('HD') & (data1['GPU'].str.contains('530')|data1['GPU'].str.contains('540')|data1['GPU'].str.contains('5300')), 'GPU'] = 'Intel HD Graphics 530'
data1.loc[data1['GPU'].str.contains('Intel')  & data1['GPU'].str.contains('Iris') , 'GPU'] = 'Intel Iris'
data1.loc[data1['GPU'].str.contains('AMD') , 'GPU'] = 'AMD Graphique'

In [32]:
data1.head()

data1 = pd.get_dummies(data1, columns=['GPU'], prefix='Grapihque')

In [33]:
data1.columns

Index(['ECRAN', 'RAM', 'POIDS', 'PRIX', 'STOCKAGESSD', 'STOCKAGEHDD',
       'STOCKAGEFlash', '_Acer', '_Apple', '_Asus', '_Dell', '_HP', '_Lenovo',
       '_MSI', '_Toshiba', 'CPU_Modifier', 'CPU_Generation', 'CPU_AMD',
       'CPU_Intel', 'Grapihque_AMD Graphique', 'Grapihque_Intel HD Graphics',
       'Grapihque_Intel HD Graphics 400', 'Grapihque_Intel HD Graphics 520',
       'Grapihque_Intel HD Graphics 530', 'Grapihque_Intel HD Graphics 620',
       'Grapihque_Intel Iris', 'Grapihque_Nvidia  GTX', 'Grapihque_Nvidia  M',
       'Grapihque_Nvidia  MX', 'Grapihque_Nvidia GeForce 920'],
      dtype='object')

In [34]:
def convert_boolean_to_binary(data1):
    for column in data1.columns:
        if data1[column].dtype == bool:
            data1[column] = data1[column].astype(int)
    return data1
data1 = convert_boolean_to_binary(data1)

In [35]:
df=pd.read_csv('DATA/Cleaned_data.csv')

In [36]:
resultat = pd.concat([data1, df], sort=False)

resultat = resultat.fillna(0)

In [37]:
resultat.head()

,ECRAN,RAM,POIDS,PRIX,STOCKAGESSD,STOCKAGEHDD,STOCKAGEFlash,_Acer,_Apple,_Asus,...,Grapihque_Nvidia MX,Grapihque_Nvidia GeForce 920,_DELL,_Hp,Unnamed: 12,ryzen,Grapihque_Intel HD Graphiques,Grapihque_Intel UHD Graphics 620,Grapihque_Nvidia Quadro,Grapihque_carte graphiques intégré
0,13.3,8,1.37,8565.441984,128.0,0.0,0.0,0.0,1.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,13.3,8,1.34,5747.462784,0.0,0.0,128.0,0.0,1.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,15.6,8,1.86,3676.320000,256.0,0.0,0.0,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,15.4,16,1.83,16223.440320,512.0,0.0,0.0,0.0,1.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,13.3,8,1.37,11531.496960,256.0,0.0,0.0,0.0,1.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [47]:
resultat.columns
resultat = resultat.drop('Unnamed: 12', axis=1, errors='ignore')


In [48]:
resultat['POIDS'].describe()

count    1520.000000
mean       15.794889
std        78.844220
min         0.000000
25%         1.640000
50%         2.100000
75%         2.620000
max      1754.000000
Name: POIDS, dtype: float64

In [49]:
resultat.head()

,ECRAN,RAM,POIDS,PRIX,STOCKAGESSD,STOCKAGEHDD,STOCKAGEFlash,_Acer,_Apple,_Asus,...,Grapihque_Nvidia M,Grapihque_Nvidia MX,Grapihque_Nvidia GeForce 920,_DELL,_Hp,ryzen,Grapihque_Intel HD Graphiques,Grapihque_Intel UHD Graphics 620,Grapihque_Nvidia Quadro,Grapihque_carte graphiques intégré
0,13.3,8,1.37,8565.441984,128.0,0.0,0.0,0.0,1.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,13.3,8,1.34,5747.462784,0.0,0.0,128.0,0.0,1.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,15.6,8,1.86,3676.320000,256.0,0.0,0.0,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,15.4,16,1.83,16223.440320,512.0,0.0,0.0,0.0,1.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,13.3,8,1.37,11531.496960,256.0,0.0,0.0,0.0,1.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [50]:
boolean_columns = [col for col in resultat.columns if resultat[col].dtype == bool]

# Create a new column based on the comparison of lowercase names
for i in range(len(boolean_columns) - 1):
    for j in range(i + 1, len(boolean_columns)):
        col1 = boolean_columns[i]
        col2 = boolean_columns[j]
        if col1.lower() == col2.lower():
            new_col_name = col1.lower() 
            resultat[new_col_name] = (resultat[col1] | resultat[col2]).astype(int)
            # Drop the original columns
            resultat = resultat.drop([col1, col2], axis=1)

In [51]:
resultat.head()

,ECRAN,RAM,POIDS,PRIX,STOCKAGESSD,STOCKAGEHDD,STOCKAGEFlash,_Acer,_Apple,_Asus,...,Grapihque_Nvidia M,Grapihque_Nvidia MX,Grapihque_Nvidia GeForce 920,_DELL,_Hp,ryzen,Grapihque_Intel HD Graphiques,Grapihque_Intel UHD Graphics 620,Grapihque_Nvidia Quadro,Grapihque_carte graphiques intégré
0,13.3,8,1.37,8565.441984,128.0,0.0,0.0,0.0,1.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,13.3,8,1.34,5747.462784,0.0,0.0,128.0,0.0,1.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,15.6,8,1.86,3676.320000,256.0,0.0,0.0,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,15.4,16,1.83,16223.440320,512.0,0.0,0.0,0.0,1.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,13.3,8,1.37,11531.496960,256.0,0.0,0.0,0.0,1.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [52]:
num_columns = len(resultat.columns)

# Display column names
column_names = resultat.columns.tolist()

# Print the results
print(f"Number of columns: {num_columns}")
print("Column names:", column_names)

Number of columns: 37
Column names: ['ECRAN', 'RAM', 'POIDS', 'PRIX', 'STOCKAGESSD', 'STOCKAGEHDD', 'STOCKAGEFlash', '_Acer', '_Apple', '_Asus', '_Dell', '_HP', '_Lenovo', '_MSI', '_Toshiba', 'CPU_Modifier', 'CPU_Generation', 'CPU_AMD', 'CPU_Intel', 'Grapihque_AMD Graphique', 'Grapihque_Intel HD Graphics', 'Grapihque_Intel HD Graphics 400', 'Grapihque_Intel HD Graphics 520', 'Grapihque_Intel HD Graphics 530', 'Grapihque_Intel HD Graphics 620', 'Grapihque_Intel Iris', 'Grapihque_Nvidia  GTX', 'Grapihque_Nvidia  M', 'Grapihque_Nvidia  MX', 'Grapihque_Nvidia GeForce 920', '_DELL', '_Hp', 'ryzen', 'Grapihque_Intel HD Graphiques', 'Grapihque_Intel UHD Graphics 620', 'Grapihque_Nvidia  Quadro', 'Grapihque_carte graphiques intégré']


In [53]:
resultat.to_csv('DATA/Final_data.csv', index=False)